# Imports

In [16]:
import pandas as pd
import sys, os, importlib
import numpy as np

from datetime import datetime
from pandas import DataFrame
from pandas import concat

pd.set_option('display.max_rows', 100)

if os.path.join('..','0_funcoes_base') not in sys.path:
    sys.path.append(os.path.join('..','0_funcoes_base')) 

df_manipulator = importlib.import_module('df_manipulator')
dt_manipulator = importlib.import_module('date_manipulator')
plot_manipulator = importlib.import_module('plot_manipulator')
file_manipulator = importlib.import_module('file_manipulator')

# Dados de configuração

In [17]:
input_config = {
    'file': {
        'ref_dir':'./out/',
        'filename':'3_export_lunar_20200517_20200620.csv',
        'delimiter': ';'
    }
}

output_config = {
    'file': {
        'ref_dir': './out',
        'delimiter':';',
        'with_header': True,
        'prefix':'4_'
    },
    'delimited_intervals': [5, 10, 20, 30, 40, 50],
    'start_region': 3,
    'end_region': 2,
    'cusum_K': 0.5
}

# Funções

In [18]:
def map_region_into_df(df, speed_bump_id, region_id, relative_start_region, relative_end_region, verbose=False):
    _df = df.copy()

    # Get indexes with speedbump
    sb_indexes_by_id  = df_manipulator.get_speed_bumps_idx(_df, speed_bump_id=speed_bump_id)
    speed_bumps       = sum(list(sb_indexes_by_id.values()), [])
    speed_bumps       = [sb.to_pydatetime() for sb in speed_bumps]

    available_min_idx = min(_df.index)
    available_max_idx = max(_df.index)

    region_by_speed_bump_timestamp = []

    for sb_id in sb_indexes_by_id:
        for sb_idx in sb_indexes_by_id[sb_id]:
            if verbose:
                print(f'Region {region_id}: [{sb_id}] {sb_idx} -> from {sb_idx-relative_start_region} to {sb_idx+relative_end_region}')

            region_by_speed_bump_timestamp.extend([dt_manipulator.get_date_window(start_date=sb_idx-relative_start_region, end_date=sb_idx+relative_end_region, bound=(available_min_idx, available_max_idx))])

    if len(region_by_speed_bump_timestamp) < 1:
        return _df

    final_region_timestamps = []
    for region_timestamp in region_by_speed_bump_timestamp:
        final_region_timestamps.extend(region_timestamp)

    final_region_timestamps = list(set(final_region_timestamps))

    _df['region_id'][_df.index.isin(final_region_timestamps)] = region_id

    return _df

def cusum(x,mean=0,K=0):
    """
    Tabular CUSUM per Montgomery,D. 1996 "Introduction to Statistical Process Control" p318
    x    : series to analyze
    mean : expected process mean
    K    : reference value, allowance, slack value-- suggest K=1/2 of the shift to be detected.
    Returns:
    x  Original series
    Cp positive CUSUM
    Cm negative CUSUM
    """
    Cp=(x*0).copy()
    Cm=Cp.copy()
    for ii in np.arange(len(x)):
        if ii == 0:
            Cp[ii]=x[ii]
            Cm[ii]=x[ii]
        else:
            Cp[ii]=np.max([0,(x[ii]-mean-K)+Cp[ii-1]])
            Cm[ii]=np.max([0,(mean-K)-x[ii]+Cm[ii-1]])
    return({'x':x, 'Cp': Cp, 'Cm': Cm})

# Carrega dataframe

In [19]:
df = df_manipulator.load_dataframes(input_config['file']['filename'], input_config['file']['ref_dir'], input_config['file']['delimiter'])
df.timestamp = pd.to_datetime(df.timestamp)
df_manipulator.set_index(df, 'timestamp', True)

df

,speed_bump_id,y,z,lat,lng
timestamp,,,,,
2020-05-17 20:14:32.800,0,8.838654,3.714386,-22.919592,-42.473961
2020-05-17 20:14:32.900,0,10.248825,3.108658,-22.919592,-42.473961
2020-05-17 20:14:33.000,0,10.842590,2.215622,-22.919592,-42.473961
2020-05-17 20:14:33.100,0,9.702957,2.493347,-22.919592,-42.473961
2020-05-17 20:14:33.200,0,9.702957,2.493347,-22.919592,-42.473961
...,...,...,...,...,...
2020-06-20 14:41:23.800,0,8.189819,6.620956,-22.871247,-42.340474
2020-06-20 14:41:23.900,0,8.189819,6.620956,-22.871247,-42.340474
2020-06-20 14:41:24.000,0,8.970322,5.715942,-22.871481,-42.340609


# Adiciona regiões nos registros

Seja *sb(t)* o instante que o veículo passou pelo quebra-mola.

Serão classificadas **dois tipos** de regiões:

- Região 0
    - Momentos da direção aonde não consta quebra-molas próximos;
    - Intervalo:  \[sb(t-inf), sb(t-3)\] V \[sb(t+2), sb(t+inf)\]
- Região 1
    - Momento que precedem quebra-mola;
    - Inclui a ação do motorista frear para quebra-mola, passar pelo mesmo e acelerar;
    - Intervalo:  \[sb(t-3), sb(t+2)\]


**Obs:** sb(t-x), aonde x possui unidade em segundo.

In [20]:
df.speed_bump_id.value_counts()

0    20532
1       84
Name: speed_bump_id, dtype: int64

In [21]:
df_out = df.copy()
df_out['region_id'] = 0

df_out = map_region_into_df(df_out, speed_bump_id=1, region_id=1, relative_start_region=pd.Timedelta(output_config['start_region'], 's'), relative_end_region=pd.Timedelta(output_config['end_region'], 's'), verbose=False)

In [22]:
df_out.region_id.value_counts()

0    16412
1     4204
Name: region_id, dtype: int64

# Adiciona normalização, média móvel, variância e diff do eixo Z e Y

In [23]:
df_out['z-norm'] = (df_out['z'] - df_out['z'].mean())/df_out['z'].std()
df_out['y-norm'] = (df_out['y'] - df_out['y'].mean())/df_out['y'].std()

for i in output_config['delimited_intervals']:    
    df_out[f'z-diff_{str(i)}'] = df_out['z-norm'].diff(periods=i)
    df_out[f'z-mean_{str(i)}'] = df_out['z-norm'].rolling(i).mean()
    df_out[f'z-std_{str(i)}']  = df_out['z-norm'].rolling(i).std()
    
    if i == 5 or i == 10:
        df_out[f'z-corr-std_{str(i)}'] = df_out['z-norm'].rolling(i).corr(df_out[f'z-std_{str(i)}'])
        df_out[f'y-corr-z-mean_{str(i)}'] = df_out['y-norm'].rolling(i).corr(df_out[f'z-mean_{str(i)}'])

# Adiciona CuSum features

In [24]:
result = cusum(df_out['z-norm'].values, mean=df_out['z-norm'].values.mean(), K = output_config['cusum_K'])

df_out['cp'] = result['Cp']
df_out['cm'] = result['Cm']

df_out['cp-norm'] = (df_out['cp'] - df_out['cp'].mean())/df_out['cp'].std()
df_out['cm-norm'] = (df_out['cm'] - df_out['cm'].mean())/df_out['cm'].std()

# Adiciona feature de distância dos pontos até a próxima região 1

In [25]:
res = np.zeros(len(df_out['region_id']))

res[0] = df_out['region_id'].iloc[0]

count = 1
for idx, val in df_out['region_id'].iloc[1:].iteritems():
    if val == 0:
        res[count] = res[count-1] + 1
    elif val == 1:
        res[count] = 0
    count +=1

df_out['region_1_distance'] = res.astype(int)

# Adiciona feature que informa a contagem dos eventos da região 1

In [26]:
res = np.zeros(len(df_out['region_id']))

res[0] = df_out['region_id'].iloc[0]

count = 0 if res[0] == 0 else 1
i = 1
for idx, val in df_out['region_id'].iloc[1:].iteritems():
    if val == 0:
        res[i] = 0
    elif val == 1:
        if res[i-1] == 0:
            count += 1
        res[i] = count
    i +=1

df_out['region_1_counter'] = res.astype(int)


In [31]:
df_out.dropna(how='any', inplace=True)

df_manipulator.apply_describe(df_out)

Shape:
 (20562, 36)
Types:

float64 :  ['y' 'z' 'lat' 'lng' 'z-norm' 'y-norm' 'z-diff_5' 'z-mean_5' 'z-std_5'
 'z-corr-std_5' 'y-corr-z-mean_5' 'z-diff_10' 'z-mean_10' 'z-std_10'
 'z-corr-std_10' 'y-corr-z-mean_10' 'z-diff_20' 'z-mean_20' 'z-std_20'
 'z-diff_30' 'z-mean_30' 'z-std_30' 'z-diff_40' 'z-mean_40' 'z-std_40'
 'z-diff_50' 'z-mean_50' 'z-std_50' 'cp' 'cm' 'cp-norm' 'cm-norm']
int64 :  ['speed_bump_id' 'region_id' 'region_1_distance' 'region_1_counter']





,null_sum,null_pct,dtypes,count,mean,median,min,max
speed_bump_id,0,0.0,int64,20562,0.004085,0.000000,0.000000,1.000000
y,0,0.0,float64,20562,9.326276,9.389313,3.047073,16.038010
z,0,0.0,float64,20562,2.944013,2.645378,-11.658783,15.675827
lat,0,0.0,float64,20562,-22.887817,-22.871247,-22.935175,-22.834573
lng,0,0.0,float64,20562,-42.405635,-42.358059,-42.495690,-42.316786
region_id,0,0.0,int64,20562,0.204455,0.000000,0.000000,1.000000
z-norm,0,0.0,float64,20562,-0.000141,-0.179973,-8.793659,7.666708
y-norm,0,0.0,float64,20562,0.000037,0.080532,-8.018070,8.570459
z-diff_5,0,0.0,float64,20562,0.000326,0.021630,-8.921498,10.301250
z-mean_5,0,0.0,float64,20562,-0.000221,-0.190785,-3.755347,3.560625


In [28]:
df_out['region_id'].value_counts()

0    16358
1     4204
Name: region_id, dtype: int64

In [29]:
df_out['speed_bump_id'].value_counts()

0    20478
1       84
Name: speed_bump_id, dtype: int64

# Salva o dataframe de saída

In [30]:
out_filename = file_manipulator.get_out_filename(output_config['file']['prefix'], output_config['file']['ref_dir'], input_config['file']['filename'])

df_out.to_csv(out_filename, sep=output_config['file']['delimiter'], header=output_config['file']['with_header'])

print(f'O arquivo {out_filename} foi gerado!')

O arquivo ./out/4_export_lunar_20200517_20200620.csv foi gerado!
